# Train with LightGBM in an Interactive Run
description: train a lightgbm model on iris data in an interactive run

This notebook is created to demonstrate how to run an experiment with all codes in a single Jupyter notebook instead of separatly creating and calling `train.py`. Although it is recommended to maintain `train.py` separatly, the way of running an experiment presented in this notebook would be useful in a very early stage of development. 

In [ ]:
%pip install -U lightgbm scikit-learn

In [ ]:
from azureml.core import Workspace
import mlflow

ws = Workspace.from_config()
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import lightgbm as lgb

# prepare train and test data
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
train_set = lgb.Dataset(X_train, label=y_train)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss

# create an experiment
mlflow.set_experiment("lightgbm-iris-interactive-example")

In [ ]:
import mlflow.lightgbm

# train model
metric_name, train_set_name = "multi_logloss", "train"

# set training parameters
params = {
    "objective": "multiclass",
    "num_class": 3,
    "learning_rate": 0.1,
    "metric": metric_name,
    "colsample_bytree": 1.0,
    "subsample": 1.0,
    "seed": 42,
}

# start a run of the experiment
with mlflow.start_run() as run:
    # enable automatic logging
    mlflow.lightgbm.autolog()

    # train the lightgbm model
    model = lgb.train(
        params,
        train_set,
        num_boost_round=10,
        valid_sets=[train_set],
        valid_names=[train_set_name],
    )

    # evaluate model
    y_proba = model.predict(X_test)
    y_pred = y_proba.argmax(axis=1)
    loss = log_loss(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)

    # log metrics from evaluation
    # run.log("log_loss", loss)
    # run.log("accuracy", acc)
    mlflow.log_metric("log_loss", loss)
    mlflow.log_metric("accuracy", acc)